# Bayesian Optimization for Hyperparameter Tuning

In [7]:
# load library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import zipfile
import urllib.request
import io
import warnings
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, KFold 
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc, roc_auc_score
warnings.filterwarnings('ignore')
#!pip install bayesian-optimization
from bayes_opt import BayesianOptimization

In [2]:
# load processed data
train_latest = pd.read_csv('https://github.com/zxyao5148/STAT3612_2023_1A_GroupProject/raw/main/data_processing/train/all_feat/train_latest.csv')
valid_latest = pd.read_csv('https://github.com/zxyao5148/STAT3612_2023_1A_GroupProject/raw/main/data_processing/valid/all_feat/valid_latest.csv')

In [3]:
# separate response and features
X_train = train_latest.drop(['id','subject_id','admittime','dischtime','readmitted_within_30days'], axis=1)
y_train = train_latest["readmitted_within_30days"]
X_test = valid_latest.drop(['id','subject_id','admittime','dischtime','readmitted_within_30days'], axis=1)
y_test = valid_latest["readmitted_within_30days"]
#from adjust_imbl import adjust_imbl
#X_train, y_train = adjust_imbl(X_train, y_train)

In [43]:
# logistic regression
def lr_objt_fun(C):
    lr = LogisticRegression(C=C, penalty=penalty, solver='liblinear', random_state=42)
    score = cross_val_score(lr, X_train, y_train, cv=5, scoring='roc_auc').mean()
    return score
pbounds = {'C': (0.001, 100)}
penalties = ['l1', 'l2']
best_score = -1
best_params = None
for penalty in penalties:
    optimizer = BayesianOptimization(f=lr_objt_fun, pbounds=pbounds, random_state=42)
    optimizer.maximize(init_points=10, n_iter=10)
    if optimizer.max['target'] > best_score:
        best_score = optimizer.max['target']
        best_params = {'C': optimizer.max['params']['C'], 'penalty': penalty}
print(f"Best result: {best_params} with score: {best_score}")

|   iter    |  target   |     C     |
-------------------------------------
| 1         | 0.7373    | 37.45     |
| 2         | 0.7368    | 95.07     |
| 3         | 0.7369    | 73.2      |
| 4         | 0.7369    | 59.87     |
| 5         | 0.7382    | 15.6      |
| 6         | 0.7382    | 15.6      |
| 7         | 0.7396    | 5.809     |
| 8         | 0.7368    | 86.62     |
| 9         | 0.7369    | 60.11     |
| 10        | 0.7369    | 70.81     |
| 11        | 0.5061    | 0.001     |
| 12        | 0.7388    | 10.11     |
| 13        | 0.7375    | 29.35     |
| 14        | 0.7371    | 45.63     |
| 15        | 0.7377    | 22.69     |
| 16        | 0.737     | 52.16     |
| 17        | 0.7368    | 80.16     |
| 18        | 0.7367    | 99.99     |
| 19        | 0.7369    | 65.77     |
| 20        | 0.7368    | 90.81     |
|   iter    |  target   |     C     |
-------------------------------------
| 1         | 0.7378    | 37.45     |
| 2         | 0.7372    | 95.07     |
| 3         

In [44]:
def lr_objt_fun(args):
    C, penalty = args
    lr = LogisticRegression(C=C, penalty=penalty, solver='liblinear', random_state=42)
    score = cross_val_score(lr, X_train, y_train, cv=5, scoring='accuracy').mean()
    return -score  
space = [
    hp.loguniform('C', np.log(0.001), np.log(100)),  
    hp.choice('penalty', ['l1','l2']) 
]
trials = Trials()
best = fmin(fn=lr_objt_fun, space=space, algo=tpe.suggest, max_evals=100, trials=trials)
print(best)

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 100/100 [15:38<00:00,  9.39s/trial, best loss: -0.849746130607094]
{'C': 0.5487068708926907, 'penalty': 0}
